# 전력망을 둘로 나누기

## 문제 설명

n개의 송전탑이 전선을 통해 하나의 트리 형태로 연결되어 있습니다. 당신은 이 전선들 중 하나를 끊어서 현재의 전력망 네트워크를 2개로 분할하려고 합니다. 이때, 두 전력망이 갖게 되는 송전탑의 개수를 최대한 비슷하게 맞추고자 합니다.

송전탑의 개수 n, 그리고 전선 정보 wires가 매개변수로 주어집니다. 전선들 중 하나를 끊어서 송전탑 개수가 가능한 비슷하도록 두 전력망으로 나누었을 때, 두 전력망이 가지고 있는 송전탑 개수의 차이(절대값)를 return 하도록 solution 함수를 완성해주세요.

## 제한사항
- n은 2 이상 100 이하인 자연수입니다.
- wires는 길이가 n-1인 정수형 2차원 배열입니다.
- wires의 각 원소는 [v1, v2] 2개의 자연수로 이루어져 있으며, 이는 전력망의 v1번 송전탑과 v2번 송전탑이 전선으로 연결되어 있다는 것을 의미합니다.
- 1 ≤ v1 < v2 ≤ n 입니다.
- 전력망 네트워크가 하나의 트리 형태가 아닌 경우는 입력으로 주어지지 않습니다.

| n  | wires                                           | result |
|----|-------------------------------------------------|--------|
| 9  | [[1,3],[2,3],[3,4],[4,5],[4,6],[4,7],[7,8],[7,9]] | 3      |
| 4  | [[1,2],[2,3],[3,4]]                             | 0      |
| 7  | [[1,2],[2,7],[3,7],[3,4],[4,5],[6,7]]           | 1      |


# 풀이

- 모든 간선에 대해
    - 간선을 하나 끊기
    - 인접 리스트 생성
    - 노드 수 찾기 -> dfs 2번

In [ ]:
from collections import defaultdict
import math

def solution(n, wires):
    min_diff = math.inf
    for i in range(len(wires)):
        new_wires = wires[0:i] + wires[i+1:]
        # 인접 리스트 생성(노드는 0부터)
        adj_list = defaultdict(list)
        for start_node, end_node in new_wires:
            adj_list[start_node - 1].append(end_node - 1)
            adj_list[end_node - 1].append(start_node - 1)
        
        visited = [False for _ in range(n)]
        counts = []
        while not all(visited):
            count = 0
            stack = []
            start_node = -1
            for node in range(n):
                if visited[node] == False:
                    start_node = node
                    break
            stack.append(start_node)
            visited[start_node] = True
            count += 1
            # dfs
            while stack:
                now_node = stack.pop()
                for adj_node in adj_list[now_node]:
                    if not visited[adj_node]:
                        stack.append(adj_node)
                        visited[adj_node] = True
                        count += 1
            counts.append(count)
        if abs(counts[0] - counts[1]) < min_diff:
            min_diff = abs(counts[0] - counts[1])
    return min_diff

이전보단 깔끔해진듯

(이전 풀이)

In [ ]:
# Disjoint Set도 괜찮을듯
# 간선 하나 없애고 그걸로 인접리스트 만들기
# => 해당 그래프를 탐색하며 노드 새기
# => 모든 노드를 탐색할 때까지
from collections import defaultdict, deque

# 간선 하나 없어진 wires가 인자로
def make_adj_list_and_search(n, wires):
    # 인접 리스트 생성
    adj_list = defaultdict(list)
    # 노드는 1번부터 시작임 => -1을 해주자
    for node1, node2 in wires:
        adj_list[node1 - 1].append(node2 - 1)
        adj_list[node2 - 1].append(node1 - 1)
    print(f"adj_list: {adj_list}")
    visited = [False for _ in range(n)]
    counts = []
    while all(visited) == False:
        count = 0
        start_node = -1
        for idx, visited_value in enumerate(visited):
            if visited_value == False: 
                start_node = idx
                break
        q = deque()
        q.append(start_node)
        visited[start_node] = True
        while q:
            now_node = q.pop() #DFS
            count += 1
            # 주위 노드 탐색
            for node in adj_list[now_node]:
                if visited[node] == False:
                    q.append(node)
                    visited[node] = True
        counts.append(count)
    #print(f'counts:{counts}')
    return abs(counts[0] - counts[1])
    
def solution(n, wires):
    diffs = []
    for idx, wire in enumerate(wires):
        diff = make_adj_list_and_search(n, wires[0:idx] + wires[idx + 1:])
        diffs.append(diff)
    return min(diffs)
    